In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics.functional import structural_similarity_index_measure, peak_signal_noise_ratio
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import math
import torchvision
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import random
import pytorch_lightning as pl
from torch.nn import Parameter
from typing import Dict, Any
from ipywidgets import IntSlider, interact

device = torch.device('mps') or torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import wandb
wandb.login()

from pytorch_lightning.loggers import WandbLogger

mps


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: simon-ma. Use `wandb login --relogin` to force relogin


In [2]:
class Dataset2D(Dataset):
    def __init__(self, data_paths, context_window=4, prediction_window=1, transform=None):
        self.data_paths = data_paths
        self.context_window = context_window
        self.prediction_window = prediction_window
        self.transform = transform
        self.samples = []
        # For every path to a volume sequence in .npy
        for data_path in self.data_paths:
            volume_seq = np.load(data_path)
            # Convert to tensor
            volume_seq = torch.from_numpy(volume_seq)
            for h in range(volume_seq.shape[1]):
                # Generate samples
                for t in range(len(volume_seq) - self.context_window - self.prediction_window + 1):
                    # Input shape: [T, C, H, W] --> [context_window, 1, 256, 256]
                    # Target shape: [T, C, H, W] --> [prediction_window, 1, 256, 256]
                    self.samples.append((volume_seq[t:t+context_window, h].unsqueeze(1), 
                                         volume_seq[t+context_window:t+context_window+prediction_window, h].unsqueeze(1)))
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):

        return self.samples[idx]
    
class Dataset3D(Dataset):
    def __init__(self, data_paths, context_window=4, prediction_window=1, transform=None):
        self.data_paths = data_paths
        self.context_window = context_window
        self.prediction_window = prediction_window
        self.transform = transform
        self.samples = []
        # For every path to a volume sequence in .npy
        for data_path in self.data_paths:
            volume_seq = np.load(data_path)
            # Convert to tensor
            volume_seq = torch.from_numpy(volume_seq)

            for t in range(volume_seq.shape[0]-context_window-prediction_window+1):
                self.samples.append([volume_seq[t:t+context_window].unsqueeze(1), 
                                     volume_seq[t+context_window:t+context_window+prediction_window].unsqueeze(1)])
                # Input-shape: [T, C, D, H, W]
                # Target-shape: [T, C, D, H, W]
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        return self.samples[idx]
    
def get_data_loaders_2D_pred(root='../Data',
                            batch_size=4, 
                            sequence_length=4, 
                            prediction_length=1, 
                            num_workers=0, 
                            pin_memory=False, 
                            drop_last=False,
                            train_split=0.8,
                            val_split=0.1):
    """
    Train dataset is 2D, Validation and Test datasets are 3D for unified validation and testing with 3D models
    """
    # Load all folder paths
    data_paths = [os.path.join(root, path) for path in os.listdir(root)]
    # Split into train/val/test
    n_train = int(len(data_paths) * train_split)
    n_val = int(len(data_paths) * val_split)
    train_paths = data_paths[:n_train]
    val_paths = data_paths[n_train:n_train+n_val]
    test_paths = data_paths[n_train+n_val:]
    
    # Create datasets
    train_dataset = Dataset2D(train_paths, sequence_length, prediction_length)
    val_dataset = Dataset3D(val_paths, sequence_length, prediction_length)
    test_dataset = Dataset3D(test_paths, sequence_length, prediction_length)
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=drop_last)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, drop_last=drop_last)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, drop_last=drop_last)
    
    return train_loader, val_loader, test_loader

2Plus1D Unet

In [10]:
class DoubleConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        
        if not mid_channels:
            mid_channels = out_channels
            
        self.double_conv = nn.Sequential(
            # First convolution
            nn.Conv2d(
                in_channels, 
                mid_channels,
                kernel_size=3,
                padding=1,
                bias=False  # No bias when using batch norm
            ),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            
            # Second convolution
            nn.Conv2d(
                mid_channels,
                out_channels,
                kernel_size=3,
                padding=1,
                bias=False  # No bias when using batch norm
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
        
        # Optional: Add residual connection if input and output channels match
        self.use_residual = in_channels == out_channels
        
    def forward(self, x):
        if self.use_residual:
            return self.double_conv(x) + x
        return self.double_conv(x)
    
class TemporalBlock(nn.Module):
    def __init__(self, channels, temporal_kernel_size=3):
        super().__init__()
        
        padding = temporal_kernel_size // 2
        
        self.temporal_conv = nn.Sequential(
            # Depthwise temporal conv
            nn.Conv1d(channels, channels,
                     kernel_size=temporal_kernel_size,
                     padding=padding,
                     groups=channels),
            nn.BatchNorm1d(channels),
            nn.ReLU(inplace=True),
            
            # Point-wise conv
            nn.Conv1d(channels, channels, kernel_size=1),
            nn.BatchNorm1d(channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        # x shape: [B, C, T, H, W]
        b, c, t, h, w = x.shape
        
        # Reshape for temporal convolution
        x_temp = x.contiguous()  # Make memory contiguous
        x_temp = x_temp.permute(0, 3, 4, 1, 2)  # [B, H, W, C, T]
        x_temp = x_temp.reshape(b*h*w, c, t)
        
        # Apply temporal convolution
        x_temp = self.temporal_conv(x_temp)
        
        # Reshape back
        x_temp = x_temp.reshape(b, h, w, c, t)
        x_temp = x_temp.permute(0, 3, 4, 1, 2)  # [B, C, T, H, W]
        
        return x_temp


class UNet2DPlusTemporal(nn.Module):
    def __init__(self, input_frames=None, output_frames=8, in_channels=1, base_filters=32):
        super(UNet2DPlusTemporal, self).__init__()
        
        self.input_frames = input_frames
        
        # Update first encoder to accept in_channels instead of hardcoded 1
        self.enc1 = DoubleConv2D(in_channels, base_filters)
        self.pool1 = nn.MaxPool2d(2)
        
        self.enc2 = DoubleConv2D(base_filters, base_filters*2)
        self.pool2 = nn.MaxPool2d(2)
        
        # Last encoder with temporal processing
        self.enc3 = DoubleConv2D(base_filters*2, base_filters*4)
        self.temporal_enc = TemporalBlock(
            channels=base_filters*4,
            temporal_kernel_size=3
        )
        self.pool3 = nn.MaxPool2d(2)
        
        # Bottleneck with temporal processing
        self.bottleneck_spatial = DoubleConv2D(base_filters*4, base_filters*8)
        self.temporal_bottleneck = TemporalBlock(
            channels=base_filters*8,
            temporal_kernel_size=3
        )
        
        # Decoder Path
        self.upconv3 = nn.ConvTranspose2d(base_filters*8, base_filters*4, 
                                         kernel_size=2, stride=2)
        self.dec3 = DoubleConv2D(base_filters*8, base_filters*4)
        
        self.upconv2 = nn.ConvTranspose2d(base_filters*4, base_filters*2, 
                                         kernel_size=2, stride=2)
        self.dec2 = DoubleConv2D(base_filters*4, base_filters*2)
        
        self.upconv1 = nn.ConvTranspose2d(base_filters*2, base_filters, 
                                         kernel_size=2, stride=2)
        self.dec1 = DoubleConv2D(base_filters*2, base_filters)
        
        self.final_conv = nn.Conv2d(base_filters, output_frames, kernel_size=1)
    
    def forward(self, x):
        # x shape: [batch, time, height, width]
        b, t, c, h, w = x.shape
        assert t == self.input_frames, f"Expected {self.input_frames} frames, got {t}"
        
        # Process each time step through initial spatial encoders
        encoder_features = []
        enc3_features = []
        
        for i in range(t):
            curr_frame = x[:, i]  # [B, C, H, W] - removed unsqueeze since we already have channels
            
            # Initial encoder path
            e1 = self.enc1(curr_frame)
            p1 = self.pool1(e1)
            
            e2 = self.enc2(p1)
            p2 = self.pool2(e2)
            
            # Store for skip connections
            encoder_features.append((e1, e2))
            
            # Last encoder
            e3 = self.enc3(p2)
            enc3_features.append(e3)
        
        # Process enc3 features temporally
        enc3_features = torch.stack(enc3_features, dim=2)  # [B, C, T, H, W]
        enc3_processed = self.temporal_enc(enc3_features)
        
        # Pool spatially after temporal processing
        b, c, t, h, w = enc3_processed.shape
        enc3_pooled = enc3_processed.contiguous()
        enc3_pooled = enc3_pooled.view(b*t, c, h, w)
        enc3_pooled = self.pool3(enc3_pooled)
        _, _, h_pooled, w_pooled = enc3_pooled.shape
        enc3_pooled = enc3_pooled.view(b, c, t, h_pooled, w_pooled)
        
        # Bottleneck processing
        bottle_features = []
        for i in range(t):
            curr_feat = enc3_pooled[:, :, i]  # [B, C, H, W]
            bottle_feat = self.bottleneck_spatial(curr_feat)
            bottle_features.append(bottle_feat)
        
        # Stack and apply temporal processing in bottleneck
        bottle_features = torch.stack(bottle_features, dim=2)  # [B, C, T, H, W]
        bottle_processed = self.temporal_bottleneck(bottle_features)
        
        # Take last temporal state for decoder
        bottle_final = bottle_processed[:, :, -1]  # [B, C, H, W]
        
        # Decoder path (using last frame's encoder features)
        e1, e2 = encoder_features[-1]
        e3 = enc3_processed[:, :, -1]
        
        d3 = self.upconv3(bottle_final)
        d3 = torch.cat([d3, e3], dim=1)
        d3 = self.dec3(d3)
        
        d2 = self.upconv2(d3)
        d2 = torch.cat([d2, e2], dim=1)
        d2 = self.dec2(d2)
        
        d1 = self.upconv1(d2)
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.dec1(d1)
        
        return self.final_conv(d1) # [B, output_frames, H, W]

In [11]:
model = UNet2DPlusTemporal(input_frames=8).to(device)
x = torch.randn(2, 8, 256, 256).to(device)
out = model(x)
out.shape

ValueError: not enough values to unpack (expected 5, got 4)

In [18]:

class Pl_Model(pl.LightningModule):
    def __init__(
        self,
        passed_model: nn.Module,
        config: Dict[str, Any],
    ):
        super(Pl_Model, self).__init__()
        self.passed_model = passed_model
        self.config = config

        #speicher alle parameter ab
        self.save_hyperparameters(ignore=["passed_model"])

        
        # Setup training components
        self.mse_criterion = nn.MSELoss()
        self.huber_criterion = nn.HuberLoss(delta=1.0)
        

    def forward(self, x):
        x = self.passed_model(x)
        return x

    def configure_optimizers(self):
        """Sets the Optimizer for the Model"""
        optimizer = optim.Adam(
            self.parameters(), 
            lr=self.config['learning_rate'],
        )
        return [optimizer]

    def _calculate_loss(self, batch, mode="train"):
        """Calculates the loss for a batch in different modes (training, validation, testing)"""
        inputs, targets = batch
        #to device
        #inputs = inputs.to(device)
        #targets = targets.to(device)
        if mode=="val" or mode=="test":
            B, T, C, D, H, W = targets.shape
            outputs = torch.zeros_like(targets)
            # account for added depth adimension in val and test sets
            for h in range(D):
                outputs[:, :, :, h] = self.forward(input[:, :, :, h]).unsqueeze(2)
        else:
            outputs = self.forward(inputs)
        mse_loss = self.mse_criterion(outputs, targets)
        huber_loss = self.huber_criterion(outputs, targets)
        total_loss = mse_loss + 0.5 * huber_loss
        #forward pass
        # mse_loss = 0.0
        # huber_loss = 0.0
        # total_loss = 0.0

        # for t in range(self.config["output_frames"]):
        #     outputs = self.forward(inputs)
            
        #     #calcualte losses
        #     mse_loss_ = self.mse_criterion(outputs, targets[:, t, :, :].unsqueeze(1))
        #     huber_loss_ = self.huber_criterion(outputs, targets[:, t, :, :].unsqueeze(1))
        #     total_loss += mse_loss_ + 0.5 * huber_loss_
        #     mse_loss += mse_loss_.item()
        #     huber_loss += huber_loss_.item()
            
        #     inputs = torch.cat([inputs[:, 1:, :, :], outputs], dim=1)       

        #logging
        self.log(f"{mode}_mse_loss", mse_loss)
        self.log(f"{mode}_huber_loss", huber_loss)
        self.log(f"{mode}_total_loss", total_loss)

        return total_loss, mse_loss, huber_loss

    def training_step(self, batch, batch_idx):
        loss, _, _ = self._calculate_loss(batch, mode="train")
        return loss

    def validation_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="val")

    def test_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="test")

    # def check_losses(self, loader, mode, use_wandb=False):
    #     mse_loss = 0.0
    #     huber_loss = 0.0
    #     total_loss = 0.0
    #     for inputs, targets in loader:
    #         for t in range(self.config["output_frames"]):
    #             mse_loss_ = self.mse_criterion(inputs[:, -1, :, :, :].unsqueeze(1), targets[:, t, :, :, :].unsqueeze(1))
    #             huber_loss_ = self.huber_criterion(inputs[:, -1, :, :, :].unsqueeze(1), targets[:, t, :, :, :].unsqueeze(1))
    #             total_loss_ = mse_loss_ + 0.5 * huber_loss_   
        
    #             mse_loss += mse_loss_.item()
    #             huber_loss += huber_loss_.item()
    #             total_loss += total_loss_.item()
    #     mse_loss = mse_loss/len(loader)
    #     huber_loss = huber_loss/len(loader)
    #     total_loss = total_loss/len(loader)
    
    #     if use_wandb:
    #         self.log(f"Checked_{mode}_mse_loss", mse_loss)
    #         self.log(f"Checked_{mode}_mse_loss", huber_loss)
    #         self.log(f"Checked_{mode}_mse_loss", total_loss)
        
    #     return mse_loss, huber_loss, total_loss
        
    def log_predictions(self):
        """Log example predictions to wandb"""
        #needs to be added to other method
        if epoch % self.config['viz_interval'] == 0:
                self.log_predictions()
        #but this whole method needs to be rewritten
        self.model.eval()
        with torch.no_grad():
            # Get a batch of validation data
            data, target = next(iter(self.val_loader))
            data = data.to(self.device)
            target = target.to(self.device)
            
            # Generate predictions
            output = self.model(data)
            
            # Log images
            wandb.log({
                "predictions": wandb.Image(output[0, 0].cpu()),
                "targets": wandb.Image(target[0, 0].cpu()),
                "input_sequence": [wandb.Image(data[0, i].cpu()) for i in range(data.shape[1])]
            })

In [20]:
config = {
    #for the dataloaders
    'batch_size': 1,
    'learning_rate': 1e-4,
    "num_workers": 4,#0, wenn die gpu nicht benutzt wird
    "pin_memory": True,#False, wenn die gpu nicht benutzt wird
    "drop_last": False,
    'epochs': 30,
    #'log_interval': 20,
    #'viz_interval': 1,
    'run_name': '2D-UNet+_temp_v3',
    'input_frames': 8,
    "output_frames": 8,
    'base_filters': 32
}

# Initialize model
model = UNet2DPlusTemporal(
    input_frames=config['input_frames'],
    base_filters=config['base_filters'],
)

# Get data loaders
train_loader, val_loader, test_loader = get_data_loaders_2D_pred(
    batch_size=config['batch_size'],
    num_workers=config["num_workers"],
    pin_memory=config["pin_memory"],
    drop_last=config["drop_last"],
    sequence_length=config["input_frames"], 
    prediction_length=config["output_frames"],
)

wandb_logger = WandbLogger(project="perfusion-ct-prediction", name=config["run_name"])

# Initialize pl_model
pl_model = Pl_Model(
    passed_model=model,
    config=config,
)
# trainer = pl.Trainer()
# trainer.test(pl_model, test_loader)
for batch_idx, (input, target) in enumerate(test_loader):
    print(input.shape)
    print(target.shape)
    pl_model.test_step((input, target), batch_idx)
    break

# # Initialize trainer
# trainer = pl.Trainer(
#     logger=wandb_logger,
#     accelerator="gpu",
#     devices= [0] if torch.cuda.is_available() else None,
#     max_epochs=config["epochs"],
# )

# wandb_logger.watch(pl_model)


torch.Size([1, 8, 1, 16, 256, 256])
torch.Size([1, 8, 1, 16, 256, 256])


/Users/simonma/miniconda3/envs/ct/lib/python3.11/site-packages/pytorch_lightning/core/module.py:441: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


In [62]:

trainer.fit(
    pl_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

#check the losses "to beat"
pl_model.check_losses(train_loader, mode="train", use_wandb=True)
pl_model.check_losses(val_loader, mode="val", use_wandb=True)
pl_model.check_losses(test_loader, mode="test", use_wandb=True)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type               | Params | Mode 
------------------------------------------------------------
0 | passed_model | UNet2DPlusTemporal | 2.0 M  | train
1 | criterion    | MSELoss            | 0      | train
2 | huber_loss   | HuberLoss          | 0      | train
------------------------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
8.047     Total estimated model params size (MB)
82        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=255` in the `DataLoader` to improve performance.


Training: |                                                                                                   …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=30` reached.


In [63]:
val_results = trainer.validate(dataloaders=val_loader)
test_results = trainer.test(dataloaders=test_loader)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.validate(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.validate(ckpt_path='best')` to use the best model or `.validate(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at ./perfusion-ct-prediction/mpw4rbc7/checkpoints/epoch=29-step=2880.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at ./perfusion-ct-prediction/mpw4rbc7/checkpoints/epoch=29-step=2880.ckpt


Validation: |                                                                                                 …

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at ./perfusion-ct-prediction/mpw4rbc7/checkpoints/epoch=29-step=2880.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at ./perfusion-ct-prediction/mpw4rbc7/checkpoints/epoch=29-step=2880.ckpt


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     val_huber_loss         0.02376500517129898
      val_mse_loss          0.04780226945877075
     val_total_loss         0.05968477204442024
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=255` in the `DataLoader` to improve performance.


Testing: |                                                                                                    …

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_huber_loss       0.011898771859705448
      test_mse_loss        0.023859838023781776
     test_total_loss       0.029809223487973213
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
save_load_path = f"../ModelWeights/{config["run_name"]}.ckpt"
trainer.save_checkpoint(save_load_path)